In [1]:
import rasterio as rio
import numpy as np
from PIL import Image

In [2]:
def load_data(paths):
    datasets = []
    for path in paths:
        with rio.open(path) as src:
            data = src.read()
            print("in here")
            data = np.squeeze(data).astype("float32")
            print("let me ooouuuut")
            datasets.append(data)
            print("k thx love you bye")
    return datasets, src

In [3]:
def get_coord_string(coords):
    if coords[0] < 0:
        lat = str(coords[0]*-1) + "S"
    else:
        lat = str(coords[0]) + "N"
    if coords[1] < 0:
        long = str(coords[1]*-1) + "W"
    else:
        long = str(coords[1]) + "E"
    coord_string = lat + long
    print(coord_string)
    
    return coord_string

In [4]:
def get_lat_long(data, src):
    print("inside lat long")
    # index array
    lat_index = np.arange(0, data.shape[0])
    long_index = np.arange(0, data.shape[1])

    print("made meshgrid :)")
   
    print("trying to transform")
    A = src.transform
    print("transformed !")
    long, lat = rio.transform.xy(A, lat_grid_flat, long_grid_flat)
    print("finished this line")
    # reshaping to shape of original data
    lat = np.array(lat).reshape(data.shape)
    print("working on this line")
    long = np.array(long).reshape(data.shape)
    print("returning")
    return lat, long

def segment_coords(data, lat, long, target_coords = (0, 0), border = 0, length = 10):
    # latitude/longitude variables
    lat_max = target_coords[0]
    lat_min = lat_max - length
    long_min = target_coords[1]
    long_max = long_min + length
    
    # length variables
    xlen = data.shape[1]
    ylen = data.shape[0]
    print("segmenting")
    # find xmin and xmax
    xmin, xmax = None, None
    for x in range(xlen):
        if long[0, x] > long_min:
            xmin = x if xmin == None else xmin
        if long[0, xlen - x - 1] < long_max:
            xmax = xlen - x - 1 if xmax == None else xmax
            
    # find ymin and ymax
    ymin, ymax = None, None
    for y in range(ylen):
        if lat[ylen - y - 1, 0] > lat_min:
            ymin = ylen - y - 1 if ymin == None else ymin
        if lat[y, 0] < lat_max:
            ymax = y if ymax == None else ymax
            
    # index data from top-bottom, left-right
    return data[ymax:ymin + border, xmin:xmax + border]


In [5]:
def segment(paths, target_coords, years, name=""):
    print("function is running !!!!")
    data, src = load_data(paths)
    lat, long = get_lat_long(data[0], src)
    for i in range(len(data)):
        print("here")
        data_seg = segment_coords(data[i], lat, long, target_coords)
        data_seg = Image.fromarray(data_seg)
        data_seg.save("../data/raw/segmented/{coords}/{name}_{yr}_{coords}.tif".format(coords = get_coord_string(target_coords), name = name, yr = years[i]))
        print("saved ", years[i], "data")

In [6]:
pop_paths = []
years = (2000,2005, 2010, 2015)
for yr in years:
    pop_data_path = "../data/raw/population/gpw_v4_population_count_rev11_{yr}_30_sec.tif".format(yr = yr)
    pop_paths.append(pop_data_path)

In [ ]:
segment(pop_paths, (-10, -60), years, name="pop")

function is running !!!!
in here
let me ooouuuut
k thx love you bye
in here
let me ooouuuut
k thx love you bye
in here
let me ooouuuut
k thx love you bye
in here
let me ooouuuut
k thx love you bye
inside lat long
made meshgrid :)
trying to transform
transformed !
